# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [42]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import HTML
from itertools import combinations
from scipy.ndimage import affine_transform
import SimpleITK as sitk


PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


/home/eddyod/programming/pipeline/src


In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import apply_affine_transform, get_affine_transformation, \
fetch_coms, list_coms, compute_affine_transformation
from library.atlas.brain_structure_manager import BrainStructureManager
from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
read_image, write_image


In [46]:
def sum_square_com(com):
    ss = np.sqrt(sum([s**2 for s in com]))
    return ss

def apply_affine_transformation(volume, matrix):
    """Apply an affine transformation to a 3D volume."""
    transformed_volume = affine_transform(volume, matrix, offset=0, order=1)
    return transformed_volume

In [60]:
moving_name = 'MD585'
fixed_name = 'MD589'

moving_all = list_coms(moving_name, scaling_factor=1)
fixed_all = list_coms(fixed_name, scaling_factor=1)

common_keys = list(moving_all.keys() & fixed_all.keys())
common_keys = list(fixed_all.keys())
print(f'{moving_name} len={len(moving_all.keys())}')
print(f'{fixed_name} len={len(fixed_all.keys())}')

print(len(common_keys))

MD585 len=51
MD589 len=51
51


In [61]:
transformation_matrix = get_affine_transformation(moving_name=moving_name, fixed_name=fixed_name, 
                                                  scaling_factor=1)


In [62]:
# these keys have very high errors: 'RtTg', 'AP'
midbrain_keys = {
            "3N_L",
            "3N_R",
            "4N_L",
            "4N_R",
            "IC",
            "PBG_L",
            "PBG_R",
            "SC",
            "SNC_L",
            "SNC_R",
            "SNR_L",
            "SNR_R",
        }
bad_keys = ('RtTg', 'AP', '3N_L', '3N_R')
bad_keys = ('RtTg', 'AP')
#bad_keys = ('RtTg',)
good_keys = set(common_keys) - set(bad_keys)
print(f'#good_keys={len(good_keys)}')
print(good_keys)

#good_keys=49
{'DC_R', 'SNC_R', '10N_L', 'VCP_L', '4N_R', 'SC', 'DC_L', '7n_L', 'Sp5O_R', '6N_L', 'PBG_R', 'Amb_L', '7N_R', 'Sp5I_L', 'Pn_R', 'LC_L', '7n_R', 'Sp5C_L', 'VCA_R', 'Pn_L', '7N_L', 'VCA_L', '4N_L', '12N', 'RMC_L', 'Sp5I_R', 'LRt_L', 'LRt_R', '5N_R', 'Sp5C_R', '3N_R', 'Tz_L', 'SNR_R', 'Sp5O_L', '5N_L', 'VCP_R', 'LC_R', 'VLL_R', 'PBG_L', 'Amb_R', 'IC', 'SNC_L', 'Tz_R', 'RMC_R', '6N_R', 'SNR_L', 'VLL_L', '10N_R', '3N_L'}


In [63]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    transformed = apply_affine_transform(moving0, transformation_matrix)
    transformed = [x for x in transformed]
    difference = [a - b for a, b in zip(transformed, fixed0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 disk to Allen RMS 237.0680595008574
# MD585 to MD589 RMS 18.2658167690059

RMS 182.569581211849


In [64]:
#transformation_matrix = np.hstack([transformation_matrix, t])
#transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])
#print(transformation_matrix)
structure = 'SC'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
#transformed_structure = apply_affine_transform(com, transformation_matrix)
print(f'{moving_name} {structure} {np.round(np.array(com))}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
print(f'{moving_name} transformed {structure} {np.round(np.array(transformed_dict[structure]))}')
#print(f'neuro res {moving_name} {structure} {np.round(com)}')


MD585 SC [10317.  3197.  4544.]
MD589 SC [11339.  4729.  4288.]
MD585 transformed SC [11268.  4742.  4387.]


In [35]:
R.shape

(3, 3)

In [41]:
params = [0.808140, -0.439742, -0.064120, 0.393423, 0.666833, -0.104813, 0.011812, 
                         0.046400, 0.993291, -240.865225, -173.912066, 34.285577]
t = np.array(params[9:])
R = np.array(params[:9]).reshape(3,3)

center = np.array([1166.5, 689.5, 436.5])
InputPoint = [ 631.000000, 884.000000, 170.000000 ]
OutputIndexFixed = [ 424, 463, 209 ]
p = np.array(InputPoint)
x1,y1,z1 = np.dot(R, p - center) + t + center
print(x1, y1, z1)
print(int(x1), int(y1), int(z1))
print(int(round(x1)), int(round(y1)), int(round(z1)))


424.43396600000005 462.54160049999996 208.77299949999997
424 462 208
424 463 209


In [ ]:
columns = ['structure', moving_name, fixed_name, 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(4)
df.sort_values(by=['sumsquares'], inplace=True)
df.to_csv('/home/eddyod/programming/pipeline/docs/sphinx/source/_static/results.csv', index=False)